In [2]:
# Import libraries
import numpy as np
import pandas as pd
import requests

# Importing Data

In [4]:
# URL of the excel file
url = 'https://www.census.gov/retail/mrts/www/mrtssales92-present.xlsx'
response = requests.get(url)
with open('data.xlsx', 'wb') as file:
    file.write(response.content)

In [5]:
# Read data
data = pd.read_excel('data.xlsx', sheet_name=None, header=None)

In [6]:
# Create dictionary containing all the dataframe 
dataframes = {sheet_name: df for sheet_name, df in data.items()}

## Automating dataframe variable assignment

In [8]:
# Dataframe variable assignment
for year, df in dataframes.items():
    globals()[f"df_{year}"] = df

## Checking df shapes

In [10]:
# Find start and end year
start_year = int(min(dataframes.keys()))
end_year = int(max(dataframes.keys()))

In [11]:
# Create list with shapes of all the df
data_summary = []

for year in range(start_year, end_year+1):
    df_name = f"df_{year}"
    df = globals()[df_name]
    data_summary.append({
        "year": year,
        "row_count": df.shape[0],
        "column_count": df.shape[1]
    })

summary_df = pd.DataFrame(data_summary)

# Data Wragling

## Automating Data Restructuring and Transformation Pipeline

In [14]:
def clean_dataframe(df, year):
    # Removing 3 rows at the top
    df = df.iloc[3:]

    # Extracting headers from columns
    col_0_1_header = df.iloc[0, :2]
    col_2_rest_header = df.iloc[1, 2:]

    # Combine the headers into one list
    headers = list(col_0_1_header) + list(col_2_rest_header)

    # Determine the number of columns to keep
    year_str = str(year)  # Convert year to string for comparison
    year_indices = [i for i, value in enumerate(headers) if year_str in value]

    if not year_indices:
        raise ValueError(f"No columns found for year: {year}")

    last_column_index = max(year_indices) + 1

    # Final headers
    final_headers = headers[:last_column_index]

    # Removing the columns with header content
    df = df.drop([3, 4]).reset_index(drop=True)

    # Finding row indices for "ADJUSTED" and "Not adjusted"
    start_row_index, end_row_index = df[df[1].str.contains("ADJUSTED", case=False, na=False)].index

    # Finding first NaN row index
    first_nan_row_index = df[df[1].isna()].index[0] if df[1].isna().any() else None

    # Extracting not adjusted sales values
    df_not_adjusted = df.iloc[start_row_index + 1:end_row_index, 0:last_column_index]
    df_not_adjusted.columns = final_headers

    # Extracting adjusted sales values
    df_adjusted = df.iloc[end_row_index + 1:first_nan_row_index, 0:last_column_index]
    df_adjusted.columns = final_headers

    # Melt the dataframes
    df_not_adjusted_melted = pd.melt(df_not_adjusted, id_vars=['NAICS  Code', 'Kind of Business'], var_name='sales_month', value_name='sales')
    df_adjusted_melted = pd.melt(df_adjusted, id_vars=['NAICS  Code', 'Kind of Business'], var_name='sales_month', value_name='sales')

    # Adding estimate_type column
    df_adjusted_melted['estimate_type'] = 'adjusted'
    df_not_adjusted_melted['estimate_type'] = 'not adjusted'

    # Union the final dataframe
    sales_combined = pd.concat([df_not_adjusted_melted, df_adjusted_melted], ignore_index=True)

    return sales_combined

# Assuming 'dataframes' is your dictionary with year keys
cleaned_dataframes = {}
for year, df in dataframes.items():
    sales_data = clean_dataframe(df, year)
    cleaned_dataframes[year] = sales_data



In [15]:
# Concating all dataframes to get single long-form table 
all_sales_data = pd.concat(cleaned_dataframes.values(), ignore_index=True)

In [16]:
# Check shape 
all_sales_data.shape

(40479, 5)

In [17]:
# Check duplicates
all_sales_data.duplicated().sum()

0

In [18]:
# Replace 'NA' and Suppressed (S) with 'nan'
all_sales_data.replace(['(NA)', '(S)'], np.nan, inplace=True)
all_sales_data = all_sales_data.infer_objects(copy=False)

C:\Users\ANJALI\AppData\Local\Temp\ipykernel_26420\415417759.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_sales_data.replace(['(NA)', '(S)'], np.nan, inplace=True)


In [19]:
# Check basic df information
all_sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40479 entries, 0 to 40478
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NAICS  Code       34977 non-null  object 
 1   Kind of Business  40479 non-null  object 
 2   sales_month       40479 non-null  object 
 3   sales             39493 non-null  float64
 4   estimate_type     40479 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [20]:
# Check number of null values
all_sales_data.isna().sum()

NAICS  Code         5502
Kind of Business       0
sales_month            0
sales                986
estimate_type          0
dtype: int64

In [21]:
# Rename the column names
all_sales_data.rename(columns={"NAICS  Code": "naics_code", "Kind of Business": "kind_of_business"}, inplace=True)

In [22]:
# Create a list of filter 
category = ["Men's clothing stores", "Women's clothing stores"]

In [23]:
# Filtering data
clothing_business_sales = all_sales_data[all_sales_data['kind_of_business'].isin(category)]

In [24]:
# Resetting index
clothing_business_sales.reset_index(drop=True, inplace=True)

In [25]:
# Dropping uneccessary columns
clothing_business_sales = clothing_business_sales.drop('naics_code', axis=1)

In [26]:
# Creating a function to deal with date format
import re
from datetime import datetime

def convert_date(date_str):
    date_str = date_str.replace(".", "")
    date_str = re.sub(r"(\b\w{3}\.?\s+\d{4}).*", r"\1", date_str)
    return datetime.strptime(date_str, "%b %Y").strftime("01/%m/%Y")

In [27]:
# Using function to change the format of sales_month
clothing_business_sales.loc[:, 'sales_month'] = clothing_business_sales['sales_month'].apply(convert_date)

In [28]:
clothing_business_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   kind_of_business  1572 non-null   object 
 1   sales_month       1572 non-null   object 
 2   sales             1486 non-null   float64
 3   estimate_type     1572 non-null   object 
dtypes: float64(1), object(3)
memory usage: 49.3+ KB


In [29]:
# Changing to datetime format
clothing_business_sales.loc[:,'sales_month'] = pd.to_datetime(clothing_business_sales['sales_month'], format='%d/%m/%Y')

In [30]:
# Sorting data based on sales_month
clothing_business_sales = clothing_business_sales.sort_values(by='sales_month', ascending=True)

In [31]:
# Convert the date string to a datetime object for comparison
cutoff_date = pd.to_datetime('2021-01-01')

# Filter based on both conditions
unadjusted_clothing_sales = clothing_business_sales[
    (clothing_business_sales['estimate_type'] == 'not adjusted') &
    (clothing_business_sales['sales_month'] < cutoff_date)
].reset_index(drop=True)

adjusted_clothing_sales = clothing_business_sales[
    (clothing_business_sales['estimate_type'] == 'adjusted') &
    (clothing_business_sales['sales_month'] < cutoff_date)
].reset_index(drop=True)

In [32]:
unadjusted_clothing_sales.drop('estimate_type', axis=1, inplace=True)
adjusted_clothing_sales.drop('estimate_type', axis=1, inplace=True)

In [33]:
unadjusted_clothing_sales.columns = unadjusted_clothing_sales.columns.str.strip()

In [34]:
nan_sales_rows = unadjusted_clothing_sales[unadjusted_clothing_sales['sales'].isna()]
nan_sales_rows

,kind_of_business,sales_month,sales
691,Men's clothing stores,2020-10-01 00:00:00,NaN
693,Men's clothing stores,2020-11-01 00:00:00,NaN


In [35]:
# Fill NaN values in 'sales' with rolling averages within each 'kind_of_business' group
unadjusted_clothing_sales['sales'] = (
    unadjusted_clothing_sales.groupby('kind_of_business', group_keys=False)['sales']
    .apply(lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean()))
)

In [36]:
adjusted_clothing_sales.columns = adjusted_clothing_sales.columns.str.strip()

In [37]:
nan_sales_rows = adjusted_clothing_sales[adjusted_clothing_sales['sales'].isna()]
nan_sales_rows

,kind_of_business,sales_month,sales
691,Men's clothing stores,2020-10-01 00:00:00,NaN
692,Men's clothing stores,2020-11-01 00:00:00,NaN


In [38]:
# Fill NaN values in 'sales' with rolling averages within each 'kind_of_business' group
adjusted_clothing_sales['sales'] = (
    adjusted_clothing_sales.groupby('kind_of_business', group_keys=False)['sales']
    .apply(lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean()))
)

# Exporting Data to Database

Since the data the is being loaded into the databse is ready for analysis and no major transformation is required for after loading, the data that is being loaded is purpose specific that's why in case of changes required to the data for the end analysis will take place here, the data will be overwritten rather than appended. Since any other form of loading will increase the complexies of checking for exceptions. 

In [122]:
import pyodbc

server = r'localhost\SQLEXPRESS' 
database = 'clothing_business_sales' 
username = r'LAPTOP-J7KF0B2O\ANJALI' 
password = '' 

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=' + server + ';'
                      'DATABASE=' + database + ';'
                      'Trusted_Connection=yes;')
cursor = cnxn.cursor()

# Delete existing data
cursor.execute("DELETE FROM clothing_business_sales.unadjusted")

# Insert Dataframe into SQL Server:
for index, row in unadjusted_clothing_sales.iterrows():
     cursor.execute("INSERT INTO clothing_business_sales.unadjusted(kind_of_business, sales_month, sales) values(?,?,?)", row['kind_of_business'], row['sales_month'], row['sales'])
cnxn.commit()
cursor.close()

In [124]:
import pyodbc

server = r'localhost\SQLEXPRESS' 
database = 'clothing_business_sales' 
username = r'LAPTOP-J7KF0B2O\ANJALI' 
password = '' 

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=' + server + ';'
                      'DATABASE=' + database + ';'
                      'Trusted_Connection=yes;')
cursor = cnxn.cursor()

# Delete existing data
cursor.execute("DELETE FROM clothing_business_sales.adjusted")

# Insert Dataframe into SQL Server:
for index, row in adjusted_clothing_sales.iterrows():
     cursor.execute("INSERT INTO clothing_business_sales.adjusted(kind_of_business, sales_month, sales) values(?,?,?)", row['kind_of_business'], row['sales_month'], row['sales'])
cnxn.commit()
cursor.close()